# Obesity, Mental Health, and Personality Traits Analysis
This notebook analyzes the relationship between obesity, mental health indicators, gender, and personality traits. The analysis includes exploratory data analysis (EDA), statistical hypothesis testing, and result interpretation.

## Load Data and Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

sns.set(style="whitegrid")

mh = pd.read_csv("/mnt/data/cleaned_mental_health.csv")
drug = pd.read_csv("/mnt/data/cleaned_drug.csv")
obesity = pd.read_csv("/mnt/data/cleaned_obesity.csv")

## Gender and Obesity Prevalence

In [ ]:
gender_obesity = obesity[obesity['Gender'].isin(['Male', 'Female'])]
male = gender_obesity[gender_obesity['Gender'] == 'Male']['ObesityRate']
female = gender_obesity[gender_obesity['Gender'] == 'Female']['ObesityRate']
t_stat, p_val = ttest_ind(male, female)
print("T-test Statistic:", t_stat)
print("P-value:", p_val)
if p_val < 0.05:
    print("Reject the null hypothesis: Significant difference between genders.")
else:
    print("Fail to reject the null hypothesis: No significant gender difference.")

In [ ]:
plt.figure(figsize=(6, 4))
sns.boxplot(data=gender_obesity, x='Gender', y='ObesityRate', palette='Set2')
plt.title("Obesity Rate by Gender")
plt.tight_layout()
plt.show()

## Growing Stress Levels by Gender

In [ ]:
plt.figure(figsize=(6, 4))
sns.countplot(data=mh, x='Gender', hue='Growing_Stress', palette='pastel')
plt.title("Growing Stress by Gender")
plt.tight_layout()
plt.show()

## Personality Traits and Mental Health Treatment

In [ ]:
df = mh.copy()
df = df[df['treatment'].isin(['Yes', 'No'])].copy()
df['treatment'] = df['treatment'].map({'Yes': 1, 'No': 0})
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])

# Merge with personality traits from drug data
df_drug = drug.copy()
df_drug = df_drug[['Nscore', 'Escore', 'Oscore', 'AScore', 'Cscore', 'Impulsive', 'SS']].reset_index(drop=True)
df = df.reset_index(drop=True).join(df_drug)

X = df[['Nscore', 'Escore', 'Oscore', 'AScore', 'Cscore', 'Impulsive', 'SS']]
y = df['treatment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

## Conclusion
- Gender does not significantly affect obesity rates based on t-test.
- Stress appears to be more frequently reported among women.
- Personality traits contribute moderately to predicting mental health treatment.
Further analysis can incorporate multi-variable interactions for deeper insights.